In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
import os
import plotly.express as px
from datetime import datetime
w=pd.read_csv("Weather.csv")
w.head(5)

,MONTH,DAY,HOUR,IATA_CODE,Temp High,Temp Low,Wind,CODE
0,1,1,0,VDO,1.0,1.0,7.0,VDO110
1,1,1,1,VDO,1.0,1.0,7.0,VDO111
2,1,1,2,VDO,1.0,1.0,7.0,VDO112
3,1,1,5,VDO,1.0,1.0,7.0,VDO115
4,1,1,6,VDO,1.0,1.0,7.0,VDO116


In [2]:
w.shape

(295187, 8)

In [3]:
delay = pd.read_csv("FLIGHT1.csv")
delay['1'],delay['HOUR']=delay['SCHEDULED_DEPARTURE'].str.split(':', 1).str
del delay['1']
delay['HOUR']=delay['HOUR'].astype(int)
delay['CODE']=delay['ORIGIN_AIRPORT']+delay['MONTH'].astype(str)+delay['DAY'].astype(str)+delay['HOUR'].astype(str)
model=delay.copy()
model

,YEAR,MONTH,DAY,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,...,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,DESTINATION_LATITUDE,DESTINATION_LONGITUDE,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DELAY,HOUR,CODE
0,2019,1,1,VJA,VDH,HAN,00:05,23:54,-11.0,21.0,...,0.0,0.0,2019-01-01,21.024500,105.841170,17.531308,106.294039,No,5,VDH115
1,2019,1,1,VJA,HAN,VDH,00:25,00:24,-1.0,11.0,...,0.0,0.0,2019-01-01,17.531308,106.294039,21.024500,105.841170,No,25,HAN1125
2,2019,1,1,BBA,UIH,KOS,00:25,00:19,-6.0,11.0,...,0.0,0.0,2019-01-01,10.622050,103.522340,13.759660,109.206123,No,25,UIH1125
3,2019,1,1,VJA,VDH,HAN,00:45,00:41,-4.0,17.0,...,0.0,0.0,2019-01-01,21.024500,105.841170,17.531308,106.294039,No,45,VDH1145
4,2019,1,1,VJA,VDH,PQC,00:50,00:46,-4.0,11.0,...,0.0,0.0,2019-01-01,9.824959,105.125893,17.531308,106.294039,No,50,VDH1150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776048,2019,12,31,VNA,VII,DAD,23:59,23:55,-4.0,22.0,...,0.0,0.0,2019-12-31,16.067780,108.220830,18.679585,105.681335,No,59,VII123159
776049,2019,12,31,VNA,SGN,UIH,23:59,23:55,-4.0,17.0,...,0.0,0.0,2019-12-31,13.759660,109.206123,10.823020,106.629650,No,59,SGN123159
776050,2019,12,31,VNA,SGN,BMV,23:59,23:50,-9.0,17.0,...,0.0,0.0,2019-12-31,12.710012,108.237752,10.823020,106.629650,No,59,SGN123159
776051,2019,12,31,VNA,CXR,BMV,23:59,23:53,-6.0,10.0,...,0.0,0.0,2019-12-31,12.710012,108.237752,12.333330,109.000000,No,59,CXR123159


In [4]:
model=model.merge(w, left_on='CODE', right_on='CODE')
model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 265786 entries, 0 to 265785
Data columns (total 38 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   YEAR                   265786 non-null  int64  
 1   MONTH_x                265786 non-null  int64  
 2   DAY_x                  265786 non-null  int64  
 3   AIRLINE                265786 non-null  object 
 4   ORIGIN_AIRPORT         265786 non-null  object 
 5   DESTINATION_AIRPORT    265786 non-null  object 
 6   SCHEDULED_DEPARTURE    265786 non-null  object 
 7   DEPARTURE_TIME         265786 non-null  object 
 8   DEPARTURE_DELAY        265786 non-null  float64
 9   TAXI_OUT               265786 non-null  float64
 10  SCHEDULED_TIME         265786 non-null  float64
 11  ELAPSED_TIME           265786 non-null  float64
 12  AIR_TIME               265786 non-null  float64
 13  DISTANCE               265786 non-null  int64  
 14  TAXI_IN                265786 non-nu

In [5]:
model=model[['DATE','YEAR','MONTH_x','DAY_x','AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT','SCHEDULED_DEPARTURE','TAXI_OUT',
             'SCHEDULED_TIME','TAXI_IN','SCHEDULED_ARRIVAL','DELAY','Temp High','Temp Low','Wind']]
model.dropna(subset=['Temp High'],inplace = True)
model=model.rename(columns={'MONTH_x': 'MONTH','DAY_x':'DAY'})
model.isnull().sum()

DATE                   0
YEAR                   0
MONTH                  0
DAY                    0
AIRLINE                0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
SCHEDULED_DEPARTURE    0
TAXI_OUT               0
SCHEDULED_TIME         0
TAXI_IN                0
SCHEDULED_ARRIVAL      0
DELAY                  0
Temp High              0
Temp Low               0
Wind                   0
dtype: int64

In [6]:
model

,DATE,YEAR,MONTH,DAY,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAXI_OUT,SCHEDULED_TIME,TAXI_IN,SCHEDULED_ARRIVAL,DELAY,Temp High,Temp Low,Wind
0,2019-01-01,2019,1,1,VJA,NCE,HAN,01:15,25.0,213.0,7.0,05:48,No,28.0,28.0,22.0
1,2019-01-01,2019,1,1,BBA,KOS,HPH,01:15,14.0,207.0,40.0,05:42,Yes,4.0,3.0,9.0
2,2019-11-01,2019,11,1,JSP,KOS,REP,07:05,13.0,145.0,6.0,10:30,No,4.0,3.0,9.0
3,2019-01-01,2019,1,1,BBA,MNL,VCL,01:59,11.0,123.0,11.0,05:02,No,18.0,16.0,9.0
4,2019-01-01,2019,1,1,BBA,MNL,VCL,01:59,11.0,123.0,11.0,05:02,No,29.0,20.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265781,2019-12-31,2019,12,31,VNA,CXR,SEA,22:12,8.0,163.0,3.0,01:55,No,29.0,23.0,17.0
265782,2019-12-31,2019,12,31,VNA,HUI,HPH,22:12,9.0,311.0,27.0,06:23,No,18.0,17.0,29.0
265783,2019-12-31,2019,12,31,VNA,SGN,DAD,23:12,16.0,64.0,5.0,00:16,No,23.0,20.0,11.0
265784,2019-12-31,2019,12,31,VJA,DIN,HAN,23:15,19.0,345.0,7.0,07:00,No,13.0,12.0,30.0


In [7]:
ori=model.groupby(['AIRLINE','ORIGIN_AIRPORT','DATE'])['YEAR'].value_counts().to_frame().rename({"YEAR": "Number of ORI_AIR"}, axis = 1).reset_index()
des=model.groupby(['AIRLINE','DESTINATION_AIRPORT','DATE'])['YEAR'].value_counts().to_frame().rename({"YEAR": "Number of DES_AIR"}, axis = 1).reset_index()


In [8]:
model['CODE']=model['AIRLINE'].astype(str)+model['ORIGIN_AIRPORT']+model['DATE'].astype(str)
ori['CODE']=ori['AIRLINE'].astype(str)+ori['ORIGIN_AIRPORT']+ori['DATE'].astype(str)
ori=ori[['CODE','Number of ORI_AIR']]
model=model.merge(ori, left_on='CODE', right_on='CODE')
del model['CODE']
model

,DATE,YEAR,MONTH,DAY,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAXI_OUT,SCHEDULED_TIME,TAXI_IN,SCHEDULED_ARRIVAL,DELAY,Temp High,Temp Low,Wind,Number of ORI_AIR
0,2019-01-01,2019,1,1,VJA,NCE,HAN,01:15,25.0,213.0,7.0,05:48,No,28.0,28.0,22.0,1
1,2019-01-01,2019,1,1,BBA,KOS,HPH,01:15,14.0,207.0,40.0,05:42,Yes,4.0,3.0,9.0,10
2,2019-01-01,2019,1,1,BBA,KOS,VCA,06:16,10.0,89.0,9.0,07:45,No,4.0,3.0,9.0,10
3,2019-01-01,2019,1,1,BBA,KOS,VCA,06:16,10.0,89.0,9.0,07:45,No,5.0,2.0,1.0,10
4,2019-01-01,2019,1,1,BBA,KOS,VCA,06:16,10.0,89.0,9.0,07:45,No,4.0,2.0,11.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265463,2019-12-31,2019,12,31,BBA,KHH,HPH,20:07,9.0,158.0,9.0,22:45,No,28.0,27.0,17.0,5
265464,2019-12-31,2019,12,31,BBA,KHH,HPH,20:07,9.0,158.0,9.0,22:45,No,21.0,19.0,22.0,5
265465,2019-12-31,2019,12,31,BBA,KHH,HPH,20:07,9.0,158.0,9.0,22:45,No,28.0,27.0,17.0,5
265466,2019-12-31,2019,12,31,BBA,KHH,HPH,20:07,9.0,158.0,9.0,22:45,No,29.0,26.0,8.0,5


In [9]:
model['CODE']=model['AIRLINE'].astype(str)+model['DESTINATION_AIRPORT']+model['DATE'].astype(str)
des['CODE']=des['AIRLINE'].astype(str)+des['DESTINATION_AIRPORT']+des['DATE'].astype(str)
des=des[['CODE','Number of DES_AIR']]
model=model.merge(des, left_on='CODE', right_on='CODE')
del model['CODE']
model

,DATE,YEAR,MONTH,DAY,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAXI_OUT,SCHEDULED_TIME,TAXI_IN,SCHEDULED_ARRIVAL,DELAY,Temp High,Temp Low,Wind,Number of ORI_AIR,Number of DES_AIR
0,2019-01-01,2019,1,1,VJA,NCE,HAN,01:15,25.0,213.0,7.0,05:48,No,28.0,28.0,22.0,1,65
1,2019-01-01,2019,1,1,VJA,VDH,HAN,02:20,15.0,200.0,8.0,06:40,No,12.0,8.0,1.0,7,65
2,2019-01-01,2019,1,1,VJA,VDH,HAN,19:10,14.0,203.0,5.0,23:33,No,14.0,4.0,5.0,7,65
3,2019-01-01,2019,1,1,VJA,VDH,HAN,16:15,10.0,200.0,4.0,20:35,No,16.0,13.0,4.0,7,65
4,2019-01-01,2019,1,1,VJA,VDO,HAN,08:25,18.0,354.0,6.0,16:19,No,-3.0,-4.0,6.0,30,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265463,2019-12-31,2019,12,31,VJA,CAN,CAH,10:13,8.0,61.0,7.0,11:14,No,29.0,23.0,18.0,2,1
265464,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,3.0,11:27,No,36.0,27.0,7.0,3,3
265465,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,3.0,11:27,No,8.0,4.0,19.0,3,3
265466,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,3.0,11:27,No,19.0,18.0,24.0,3,3


In [10]:
ta=model.groupby(['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT','DATE'])[['TAXI_IN','TAXI_OUT']].mean().rename({"TAXI_IN": "MEAN OF TAXI_IN",'TAXI_OUT':'MEAN OF TAXI_OUT'}, axis = 1).reset_index()
model['CODE']=model['AIRLINE'].astype(str)+model['ORIGIN_AIRPORT']+model['DESTINATION_AIRPORT']+model['DATE'].astype(str)
ta['CODE']=ta['AIRLINE'].astype(str)+ta['ORIGIN_AIRPORT']+ta['DESTINATION_AIRPORT']+ta['DATE'].astype(str)
ta=ta[['CODE','MEAN OF TAXI_IN','MEAN OF TAXI_OUT']]
model=model.merge(ta, left_on='CODE', right_on='CODE')
del model['CODE']
model

,DATE,YEAR,MONTH,DAY,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAXI_OUT,SCHEDULED_TIME,TAXI_IN,SCHEDULED_ARRIVAL,DELAY,Temp High,Temp Low,Wind,Number of ORI_AIR,Number of DES_AIR,MEAN OF TAXI_IN,MEAN OF TAXI_OUT
0,2019-01-01,2019,1,1,VJA,NCE,HAN,01:15,25.0,213.0,7.0,05:48,No,28.0,28.0,22.0,1,65,7.000000,25.0
1,2019-01-01,2019,1,1,VJA,VDH,HAN,02:20,15.0,200.0,8.0,06:40,No,12.0,8.0,1.0,7,65,5.666667,13.0
2,2019-01-01,2019,1,1,VJA,VDH,HAN,19:10,14.0,203.0,5.0,23:33,No,14.0,4.0,5.0,7,65,5.666667,13.0
3,2019-01-01,2019,1,1,VJA,VDH,HAN,16:15,10.0,200.0,4.0,20:35,No,16.0,13.0,4.0,7,65,5.666667,13.0
4,2019-01-01,2019,1,1,VJA,VDO,HAN,08:25,18.0,354.0,6.0,16:19,No,-3.0,-4.0,6.0,30,65,7.000000,27.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265463,2019-12-31,2019,12,31,VJA,CAN,CAH,10:13,8.0,61.0,7.0,11:14,No,29.0,23.0,18.0,2,1,7.000000,8.0
265464,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,3.0,11:27,No,36.0,27.0,7.0,3,3,3.000000,10.0
265465,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,3.0,11:27,No,8.0,4.0,19.0,3,3,3.000000,10.0
265466,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,3.0,11:27,No,19.0,18.0,24.0,3,3,3.000000,10.0


In [11]:
model['DEPARTURE']=model['SCHEDULED_DEPARTURE'].str.replace(':','').astype(int)
model['ARRIVAL']=model['SCHEDULED_ARRIVAL'].str.replace(':','').astype(int)
model

,DATE,YEAR,MONTH,DAY,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAXI_OUT,SCHEDULED_TIME,...,DELAY,Temp High,Temp Low,Wind,Number of ORI_AIR,Number of DES_AIR,MEAN OF TAXI_IN,MEAN OF TAXI_OUT,DEPARTURE,ARRIVAL
0,2019-01-01,2019,1,1,VJA,NCE,HAN,01:15,25.0,213.0,...,No,28.0,28.0,22.0,1,65,7.000000,25.0,115,548
1,2019-01-01,2019,1,1,VJA,VDH,HAN,02:20,15.0,200.0,...,No,12.0,8.0,1.0,7,65,5.666667,13.0,220,640
2,2019-01-01,2019,1,1,VJA,VDH,HAN,19:10,14.0,203.0,...,No,14.0,4.0,5.0,7,65,5.666667,13.0,1910,2333
3,2019-01-01,2019,1,1,VJA,VDH,HAN,16:15,10.0,200.0,...,No,16.0,13.0,4.0,7,65,5.666667,13.0,1615,2035
4,2019-01-01,2019,1,1,VJA,VDO,HAN,08:25,18.0,354.0,...,No,-3.0,-4.0,6.0,30,65,7.000000,27.5,825,1619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265463,2019-12-31,2019,12,31,VJA,CAN,CAH,10:13,8.0,61.0,...,No,29.0,23.0,18.0,2,1,7.000000,8.0,1013,1114
265464,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,...,No,36.0,27.0,7.0,3,3,3.000000,10.0,1106,1127
265465,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,...,No,8.0,4.0,19.0,3,3,3.000000,10.0,1106,1127
265466,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,...,No,19.0,18.0,24.0,3,3,3.000000,10.0,1106,1127


In [12]:
a=model.groupby(['DATE','AIRLINE','DESTINATION_AIRPORT','ARRIVAL','DELAY'])['YEAR'].count().to_frame().rename({"YEAR": "Number of DELAY"}, axis = 1).reset_index()
a=a.sort_values(['DATE','AIRLINE','DESTINATION_AIRPORT','ARRIVAL','Number of DELAY'], ascending=False)
a.drop_duplicates(['DATE','AIRLINE','DESTINATION_AIRPORT','ARRIVAL'],keep='first',inplace=True)
a['CODE']=a['AIRLINE'].astype(str)+a['DESTINATION_AIRPORT']+a['DATE'].astype(str)
a

,DATE,AIRLINE,DESTINATION_AIRPORT,ARRIVAL,DELAY,Number of DELAY,CODE
192227,2019-12-31,VTV,VII,1950,No,4,VTVVII2019-12-31
192226,2019-12-31,VTV,VII,1945,No,1,VTVVII2019-12-31
192225,2019-12-31,VTV,VII,1620,No,5,VTVVII2019-12-31
192224,2019-12-31,VTV,VII,1500,No,5,VTVVII2019-12-31
192223,2019-12-31,VTV,VII,1415,Yes,5,VTVVII2019-12-31
...,...,...,...,...,...,...,...
4,2019-01-01,BBA,BMV,2202,No,5,BBABMV2019-01-01
3,2019-01-01,BBA,BMV,1637,No,1,BBABMV2019-01-01
2,2019-01-01,BBA,BMV,1425,No,4,BBABMV2019-01-01
1,2019-01-01,BBA,BLR,2259,No,4,BBABLR2019-01-01


In [13]:
a.DELAY.value_counts()

No     153518
Yes     38399
Name: DELAY, dtype: int64

In [14]:
a['ARRIVAL'].unique()

array([1950, 1945, 1620, ...,  123,  119,  336], dtype=int64)

In [15]:
b=model.groupby(['DATE','AIRLINE','ORIGIN_AIRPORT','DEPARTURE']).count().reset_index()
b=b[['DATE','AIRLINE','ORIGIN_AIRPORT','DEPARTURE']]
b['PREDELAY']=None
b['CODE']=b['AIRLINE'].astype(str)+b['ORIGIN_AIRPORT']+b['DATE'].astype(str)
b

,DATE,AIRLINE,ORIGIN_AIRPORT,DEPARTURE,PREDELAY,CODE
0,2019-01-01,BBA,BLR,810,None,BBABLR2019-01-01
1,2019-01-01,BBA,BLR,1016,None,BBABLR2019-01-01
2,2019-01-01,BBA,BMV,1515,None,BBABMV2019-01-01
3,2019-01-01,BBA,BMV,1729,None,BBABMV2019-01-01
4,2019-01-01,BBA,CAH,719,None,BBACAH2019-01-01
...,...,...,...,...,...,...
179292,2019-12-31,VTV,VII,1100,None,VTVVII2019-12-31
179293,2019-12-31,VTV,VII,1200,None,VTVVII2019-12-31
179294,2019-12-31,VTV,VII,1210,None,VTVVII2019-12-31
179295,2019-12-31,VTV,VII,1610,None,VTVVII2019-12-31


In [17]:
b=pd.read_csv("b.csv")
b.isnull().sum()
b

,CODE,PREDELAY
0,BBABLR2019-01-01,No
1,BBABLR2019-01-01,No
2,BBABMV2019-01-01,No
3,BBABMV2019-01-01,No
4,BBACAH2019-01-01,No
...,...,...
179325,VTVVII2019-12-31,No
179326,VTVVII2019-12-31,No
179327,VTVVII2019-12-31,No
179328,VTVVII2019-12-31,No


In [19]:
model['CODE']=model['AIRLINE'].astype(str)+model['ORIGIN_AIRPORT']+model['DATE'].astype(str)
model=model.merge(b, left_on='CODE', right_on='CODE')
del model['CODE']
model.drop_duplicates(keep='first',inplace=True)
model

,DATE,YEAR,MONTH,DAY,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,TAXI_OUT,SCHEDULED_TIME,...,Temp High,Temp Low,Wind,Number of ORI_AIR,Number of DES_AIR,MEAN OF TAXI_IN,MEAN OF TAXI_OUT,DEPARTURE,ARRIVAL,PREDELAY
0,2019-01-01,2019,1,1,VJA,NCE,HAN,01:15,25.0,213.0,...,28.0,28.0,22.0,1,65,7.000000,25.0,115,548,No
1,2019-01-01,2019,1,1,VJA,VDH,HAN,02:20,15.0,200.0,...,12.0,8.0,1.0,7,65,5.666667,13.0,220,640,No
8,2019-01-01,2019,1,1,VJA,VDH,HAN,19:10,14.0,203.0,...,14.0,4.0,5.0,7,65,5.666667,13.0,1910,2333,No
15,2019-01-01,2019,1,1,VJA,VDH,HAN,16:15,10.0,200.0,...,16.0,13.0,4.0,7,65,5.666667,13.0,1615,2035,No
22,2019-01-01,2019,1,1,VJA,VDH,UIH,23:20,16.0,310.0,...,12.0,8.0,1.0,7,10,7.000000,16.0,2320,530,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756756,2019-12-30,2019,12,30,JSP,AMD,UIH,21:15,18.0,180.0,...,22.0,20.0,14.0,1,1,10.000000,18.0,2115,2215,No
4756757,2019-12-30,2019,12,30,JSP,ROM,HND,21:15,21.0,93.0,...,29.0,26.0,32.0,1,1,8.000000,21.0,2115,2248,Yes
4756758,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,...,36.0,27.0,7.0,3,3,3.000000,10.0,1106,1127,No
4756759,2019-12-31,2019,12,31,VJA,VKG,BMV,11:06,10.0,21.0,...,8.0,4.0,19.0,3,3,3.000000,10.0,1106,1127,No


In [20]:
model.isnull().sum()

DATE                   0
YEAR                   0
MONTH                  0
DAY                    0
AIRLINE                0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
SCHEDULED_DEPARTURE    0
TAXI_OUT               0
SCHEDULED_TIME         0
TAXI_IN                0
SCHEDULED_ARRIVAL      0
DELAY                  0
Temp High              0
Temp Low               0
Wind                   0
Number of ORI_AIR      0
Number of DES_AIR      0
MEAN OF TAXI_IN        0
MEAN OF TAXI_OUT       0
DEPARTURE              0
ARRIVAL                0
PREDELAY               0
dtype: int64

In [29]:
export_csv = model.to_csv (r'Model.csv', index = None, header=True)

In [30]:
model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 346826 entries, 0 to 4756760
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   DATE                 346826 non-null  object 
 1   YEAR                 346826 non-null  int64  
 2   MONTH                346826 non-null  int64  
 3   DAY                  346826 non-null  int64  
 4   AIRLINE              346826 non-null  object 
 5   ORIGIN_AIRPORT       346826 non-null  object 
 6   DESTINATION_AIRPORT  346826 non-null  object 
 7   SCHEDULED_DEPARTURE  346826 non-null  object 
 8   TAXI_OUT             346826 non-null  float64
 9   SCHEDULED_TIME       346826 non-null  float64
 10  TAXI_IN              346826 non-null  float64
 11  SCHEDULED_ARRIVAL    346826 non-null  object 
 12  DELAY                346826 non-null  object 
 13  Temp High            346826 non-null  float64
 14  Temp Low             346826 non-null  float64
 15  Wind            